In [7]:
from google.colab import drive
drive.mount('/content/drive')
file_link = "/content/drive/MyDrive/NCKH/NCKH - #AI/Code/final/EVRPTW_Data/dataBVfinal.json"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import numpy as np
import matplotlib.pyplot as plt
import json

In [9]:
with open(file_link, 'r', encoding="UTF-8") as file:
    data = json.load(file)

In [11]:
data["COORDINATES"]

{'1': [10.756464, 106.66593],
 '2': [10.790984, 106.653332],
 '3': [10.755638, 106.664679],
 '4': [10.753905, 106.67201],
 '5': [10.753009, 106.678587],
 '6': [10.774047, 106.681291],
 '7': [10.754124, 106.678247],
 '8': [10.772002, 106.699546],
 '9': [10.75614, 106.661673],
 '10': [10.778621, 106.684976],
 '11': [10.756814, 106.675074],
 '12': [10.755491, 106.670154],
 '13': [10.774411, 106.666582],
 '14': [10.756935, 106.659699],
 '15': [10.804078, 106.694171],
 '16': [10.768803, 106.670417],
 '17': [10.781435, 106.702952],
 '18': [10.758443, 106.664146],
 '19': [10.768677, 106.685872],
 '20': [10.805024, 106.694278],
 '21': [10.772943, 106.667099],
 '22': [10.79799, 106.696706],
 '23': [10.86439, 106.745565],
 '24': [10.783498, 106.641932],
 '25': [10.746516, 106.634586],
 '26': [10.782242, 106.768492],
 '27': [10.765302, 106.702236],
 '28': [10.741707, 106.676466],
 '29': [10.771043, 106.659919],
 '30': [10.784302, 106.684059],
 '31': [10.764448, 106.60322],
 '32': [10.975346, 106.

In [49]:
import folium

# Create a map centered around Ho Chi Minh City
hcmc_map = folium.Map(location=[10.7769, 106.6957], zoom_start=12)

# Iterate through the coordinates and add markers
for node_id, coordinates in data["COORDINATES"].items():
    node_id = int(node_id)
    latitude = coordinates[0]
    longitude = coordinates[1]

    if node_id == 1:
        # Node 1: Marker màu đỏ (định vị quan trọng)
        folium.Marker(
            location=[latitude, longitude],
            icon=folium.Icon(color='red', icon='map-marker'),
            popup=f"Node {node_id}"
        ).add_to(hcmc_map)

    elif 2 <= node_id <= 36:
        # Node 2 - 36: Marker xanh dương (biểu tượng định vị GPS)
        folium.Marker(
            location=[latitude, longitude],
            icon=folium.Icon(color='blue', icon='map-marker'),
            popup=f"Node {node_id}"
        ).add_to(hcmc_map)

    elif 37 <= node_id <= 42:
        # Node 37 - 42: Marker xanh lá (biểu tượng định vị khác)
        folium.Marker(
            location=[latitude, longitude],
            icon=folium.Icon(color='green', icon='map-marker'),
            popup=f"Node {node_id}"
        ).add_to(hcmc_map)

    else:
        # Các node còn lại (mặc định)
        folium.Marker(
            location=[latitude, longitude],
            popup=f"Node {node_id}"
        ).add_to(hcmc_map)

# Hiển thị bản đồ
hcmc_map


In [27]:
# Save the map to an HTML file
hcmc_map.save("/content/drive/MyDrive/NCKH/NCKH - #AI/Code/final/hcmc_map.html")

In [50]:

# Routes to draw
routes = [[1, 4, 28, 35, 1],
 [1, 3, 7, 26, 30, 1],
 [1, 9, 12, 5, 22, 15, 34, 38, 32, 1],
 [1, 2, 33, 24, 1],
 [1, 10, 11, 13, 18, 29, 14, 25, 1],
 [1, 21, 6, 27, 19, 36, 31, 1],
 [1, 8, 17, 23, 20, 16, 1]]


# # Draw the routes
# for route in routes:
#     locations = []
#     for node_id in route:
#         locations.append(data["COORDINATES"][str(node_id)])  # Access coordinates using string key

#     folium.PolyLine(locations=locations, color="red", weight=2.5, opacity=1).add_to(hcmc_map)

# # Display the map
# hcmc_map

# # Save the map to an HTML file (optional)
# hcmc_map.save("/content/drive/MyDrive/NCKH/NCKH - #AI/Code/final/hcmc_map_with_routes.html")

In [ ]:
import requests
import folium

def get_route_from_osrm(coordinates):
    """
    Gets the route between coordinates using OSRM API.
    """
    # OSRM API endpoint
    osrm_url = "http://router.project-osrm.org/route/v1/driving/"

    # Construct the coordinates string for the OSRM API request
    coordinates_str = ";".join([f"{lon},{lat}" for lat, lon in coordinates])

    # Construct the full API request URL
    full_url = f"{osrm_url}{coordinates_str}?overview=full&geometries=geojson"

    try:
        response = requests.get(full_url)
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json()

        # Extract route geometry
        route_geometry = data["routes"][0]["geometry"]["coordinates"]

        # Convert coordinates to latitude, longitude format
        route_coordinates = [[coord[1], coord[0]] for coord in route_geometry]
        return route_coordinates

    except requests.exceptions.RequestException as e:
        print(f"Error fetching route from OSRM: {e}")
        return None


# Draw the routes using OSRM
for route in routes:
    locations = []
    for node_id in route:
        locations.append(data["COORDINATES"][str(node_id)])  # Access coordinates using string key

    # Use OSRM to get the actual route
    osrm_route = get_route_from_osrm(locations)

    if osrm_route:
        folium.PolyLine(locations=osrm_route, color="blue", weight=2.5, opacity=1).add_to(hcmc_map)
    else:
        # Fallback to drawing a simple polyline if OSRM request fails
        folium.PolyLine(locations=locations, color="red", weight=2.5, opacity=1).add_to(hcmc_map)

# Display the map
hcmc_map

In [52]:
# Save the map to an HTML file (optional)
hcmc_map.save("/content/drive/MyDrive/NCKH/NCKH - #AI/Code/final/hcmc_map_with_real_routes_3.html")